In [1]:
import time
import pennylane as qml
from qulacs import QuantumCircuit, QuantumState, QuantumStateGpu

N_WIRES = 6
N_ITERS = 300    # 300 is enough for comparison


print("=== Running 6-Qubit Benchmarks ===")

########################################
# Build a reusable 6-qubit circuit
########################################
def build_qulacs_circuit(n):
    qc = QuantumCircuit(n)
    for i in range(n):
        qc.add_RX_gate(i, 0.3)
        qc.add_RZ_gate(i, 0.4)
    for i in range(n - 1):
        qc.add_CNOT_gate(i, i + 1)
    return qc

qc = build_qulacs_circuit(N_WIRES)


########################################
# 1) QULACS CPU
########################################
state_cpu = QuantumState(N_WIRES)

t0 = time.time()
for _ in range(N_ITERS):
    state_cpu.set_zero_state()
    qc.update_quantum_state(state_cpu)
t_qulacs_cpu = time.time() - t0

print(f"Qulacs CPU: {t_qulacs_cpu:.4f} s")


########################################
# 2) QULACS GPU
########################################
state_gpu = QuantumStateGpu(N_WIRES)

t0 = time.time()
for _ in range(N_ITERS):
    state_gpu.set_zero_state()
    qc.update_quantum_state(state_gpu)
t_qulacs_gpu = time.time() - t0

print(f"Qulacs GPU: {t_qulacs_gpu:.4f} s")


########################################
# 3) PENNYLANE default.qubit (CPU)
########################################
dev_default = qml.device("default.qubit", wires=N_WIRES)

@qml.qnode(dev_default)
def pl_default():
    for i in range(N_WIRES):
        qml.RX(0.3, wires=i)
        qml.RZ(0.4, wires=i)
    for i in range(N_WIRES - 1):
        qml.CNOT(wires=[i, i + 1])
    return qml.state()

t0 = time.time()
for _ in range(N_ITERS):
    pl_default()
t_pl_default = time.time() - t0

print(f"PL default.qubit CPU: {t_pl_default:.4f} s")


########################################
# 4) PENNYLANE lightning.qubit (CPU)
########################################
dev_lightning = qml.device("lightning.qubit", wires=N_WIRES)

@qml.qnode(dev_lightning)
def pl_lightning():
    for i in range(N_WIRES):
        qml.RX(0.3, wires=i)
        qml.RZ(0.4, wires=i)
    for i in range(N_WIRES - 1):
        qml.CNOT(wires=[i, i + 1])
    return qml.state()

t0 = time.time()
for _ in range(N_ITERS):
    pl_lightning()
t_pl_lightning = time.time() - t0

print(f"PL lightning.qubit CPU: {t_pl_lightning:.4f} s")


########################################
# SUMMARY
########################################
print("\n=== SUMMARY (6 qubits) ===")
print(f"Qulacs CPU:               {t_qulacs_cpu:.4f} s")
print(f"Qulacs GPU:               {t_qulacs_gpu:.4f} s")
print(f"PL default.qubit CPU:     {t_pl_default:.4f} s")
print(f"PL lightning.qubit CPU:   {t_pl_lightning:.4f} s")

=== Running 6-Qubit Benchmarks ===
Qulacs CPU: 0.0015 s
Qulacs GPU: 0.0341 s
PL default.qubit CPU: 0.5054 s
PL lightning.qubit CPU: 0.2545 s

=== SUMMARY (6 qubits) ===
Qulacs CPU:               0.0015 s
Qulacs GPU:               0.0341 s
PL default.qubit CPU:     0.5054 s
PL lightning.qubit CPU:   0.2545 s


In [4]:
import numpy as np
import time
from qulacs import QuantumCircuit, QuantumState, Observable
from qulacs import QuantumStateGpu  # GPU state
import pennylane as qml

# ==========================================================
# Settings
# ==========================================================
N = 6
STEPS = 40
LR = 0.1
EPS = 1e-4          # finite difference
TARGET = 1.0

# Shared ansatz parameters
np.random.seed(0)
INIT_PARAMS = np.random.uniform(-0.1, 0.1, size=2*N)

# ==========================================================
# Helper: Classical finite-difference gradient
# ==========================================================
def finite_diff(eval_fn, params):
    grad = np.zeros_like(params)
    for i in range(len(params)):
        p1 = params.copy(); p1[i] += EPS
        p2 = params.copy(); p2[i] -= EPS
        grad[i] = (eval_fn(p1) - eval_fn(p2)) / (2*EPS)
    return grad

# ==========================================================
# 1) QULACS CPU
# ==========================================================
obs = Observable(N)
obs.add_operator(1.0, "Z 0")

def qulacs_eval_cpu(params):
    qc = QuantumCircuit(N)
    idx = 0
    for q in range(N):
        qc.add_RX_gate(q, params[idx]); idx+=1
        qc.add_RY_gate(q, params[idx]); idx+=1
    for q in range(N-1):
        qc.add_CNOT_gate(q,q+1)

    st = QuantumState(N)
    st.set_zero_state()
    qc.update_quantum_state(st)
    return obs.get_expectation_value(st)

def train_qulacs_cpu():
    params = INIT_PARAMS.copy()
    t0 = time.time()
    for s in range(STEPS):
        val = qulacs_eval_cpu(params)
        loss = (TARGET - val)**2
        grad = finite_diff(qulacs_eval_cpu, params)
        params -= LR * grad
    return time.time() - t0, val

# ==========================================================
# 2) QULACS GPU
# ==========================================================
def qulacs_eval_gpu(params):
    qc = QuantumCircuit(N)
    idx = 0
    for q in range(N):
        qc.add_RX_gate(q, params[idx]); idx+=1
        qc.add_RY_gate(q, params[idx]); idx+=1
    for q in range(N-1):
        qc.add_CNOT_gate(q,q+1)

    st = QuantumStateGpu(N)
    st.set_zero_state()
    qc.update_quantum_state(st)
    return obs.get_expectation_value(st)

def train_qulacs_gpu():
    params = INIT_PARAMS.copy()
    t0 = time.time()
    for s in range(STEPS):
        val = qulacs_eval_gpu(params)
        loss = (TARGET - val)**2
        grad = finite_diff(qulacs_eval_gpu, params)
        params -= LR * grad
    return time.time() - t0, val

# ==========================================================
# 3) PennyLane default.qubit CPU
# ==========================================================
dev_default = qml.device("default.qubit", wires=N)

@qml.qnode(dev_default)
def pl_circuit_default(params):
    idx=0
    for q in range(N):
        qml.RX(params[idx], q); idx+=1
        qml.RY(params[idx], q); idx+=1
    for q in range(N-1):
        qml.CNOT([q,q+1])
    return qml.expval(qml.PauliZ(0))

def train_pl_default():
    params = INIT_PARAMS.copy()
    t0 = time.time()
    for s in range(STEPS):
        val = pl_circuit_default(params)
        loss = (TARGET - val)**2
        grad = finite_diff(pl_circuit_default, params)
        params -= LR * grad
    return time.time() - t0, float(val)

# ==========================================================
# 4) PennyLane lightning.qubit CPU
# ==========================================================
dev_lightning = qml.device("lightning.qubit", wires=N)

@qml.qnode(dev_lightning)
def pl_circuit_lightning(params):
    idx=0
    for q in range(N):
        qml.RX(params[idx], q); idx+=1
        qml.RY(params[idx], q); idx+=1
    for q in range(N-1):
        qml.CNOT([q,q+1])
    return qml.expval(qml.PauliZ(0))

def train_pl_lightning():
    params = INIT_PARAMS.copy()
    t0 = time.time()
    for s in range(STEPS):
        val = pl_circuit_lightning(params)
        loss = (TARGET - val)**2
        grad = finite_diff(pl_circuit_lightning, params)
        params -= LR * grad
    return time.time() - t0, float(val)

# ==========================================================
# RUN ALL 4 TRAININGS
# ==========================================================
print("\n=== Running 6-qubit VQC training (40 steps) ===\n")

t_qcpu, v_qcpu = train_qulacs_cpu()
print(f"Qulacs CPU:             {t_qcpu:.4f} s  | final <Z0> = {v_qcpu:.4f}")

t_qgpu, v_qgpu = train_qulacs_gpu()
print(f"Qulacs GPU:             {t_qgpu:.4f} s  | final <Z0> = {v_qgpu:.4f}")

t_pdef, v_pdef = train_pl_default()
print(f"PL default.qubit CPU:   {t_pdef:.4f} s  | final <Z0> = {v_pdef:.4f}")

t_plight, v_plight = train_pl_lightning()
print(f"PL lightning.qubit CPU: {t_plight:.4f} s  | final <Z0> = {v_plight:.4f}")

print("\n=== SUMMARY ===")
print(f"Qulacs CPU:             {t_qcpu:.4f} s")
print(f"Qulacs GPU:             {t_qgpu:.4f} s")
print(f"PL default.qubit CPU:   {t_pdef:.4f} s")
print(f"PL lightning.qubit CPU: {t_plight:.4f} s")


=== Running 6-qubit VQC training (40 steps) ===

Qulacs CPU:             0.0427 s  | final <Z0> = 0.1132
Qulacs GPU:             0.3437 s  | final <Z0> = 0.1132
PL default.qubit CPU:   1.6647 s  | final <Z0> = 0.1132
PL lightning.qubit CPU: 0.9654 s  | final <Z0> = 0.1132

=== SUMMARY ===
Qulacs CPU:             0.0427 s
Qulacs GPU:             0.3437 s
PL default.qubit CPU:   1.6647 s
PL lightning.qubit CPU: 0.9654 s


In [5]:
import numpy as np
import time
from qulacs import QuantumCircuit, QuantumState, Observable, QuantumStateGpu
import pennylane as qml

LR = 0.05
EPS = 1e-4
STEPS = 25   # reduce steps because costs grow quickly

def finite_diff(eval_fn, params):
    grad = np.zeros_like(params)
    for i in range(len(params)):
        p1 = params.copy(); p1[i]+=EPS
        p2 = params.copy(); p2[i]-=EPS
        grad[i] = (eval_fn(p1)-eval_fn(p2))/(2*EPS)
    return grad

def build_ansatz(N, params):
    qc = QuantumCircuit(N)
    idx=0
    for q in range(N):
        qc.add_RX_gate(q, params[idx]); idx+=1
        qc.add_RY_gate(q, params[idx]); idx+=1
    for q in range(N-1):
        qc.add_CNOT_gate(q, q+1)
    return qc

def train_qulacs_cpu(N, params):
    obs = Observable(N); obs.add_operator(1.0,"Z 0")
    def eval_fn(p):
        qc = build_ansatz(N,p)
        st = QuantumState(N); st.set_zero_state()
        qc.update_quantum_state(st)
        return obs.get_expectation_value(st)
    t0=time.time()
    p=params.copy()
    for _ in range(STEPS):
        grad=finite_diff(eval_fn,p)
        p-=LR*grad
    return time.time()-t0

def train_qulacs_gpu(N, params):
    obs = Observable(N); obs.add_operator(1.0,"Z 0")
    def eval_fn(p):
        qc = build_ansatz(N,p)
        st = QuantumStateGpu(N); st.set_zero_state()
        qc.update_quantum_state(st)
        return obs.get_expectation_value(st)
    t0=time.time()
    p=params.copy()
    for _ in range(STEPS):
        grad=finite_diff(eval_fn,p)
        p-=LR*grad
    return time.time()-t0

def train_pl_default(N, params):
    dev = qml.device("default.qubit", wires=N)
    @qml.qnode(dev)
    def circuit(p):
        idx=0
        for q in range(N):
            qml.RX(p[idx],q); idx+=1
            qml.RY(p[idx],q); idx+=1
        for q in range(N-1):
            qml.CNOT([q,q+1])
        return qml.expval(qml.PauliZ(0))
    t0=time.time()
    p=params.copy()
    for _ in range(STEPS):
        grad=finite_diff(circuit,p)
        p-=LR*grad
    return time.time()-t0

def train_pl_lightning(N, params):
    dev = qml.device("lightning.qubit", wires=N)
    @qml.qnode(dev)
    def circuit(p):
        idx=0
        for q in range(N):
            qml.RX(p[idx],q); idx+=1
            qml.RY(p[idx],q); idx+=1
        for q in range(N-1):
            qml.CNOT([q,q+1])
        return qml.expval(qml.PauliZ(0))
    t0=time.time()
    p=params.copy()
    for _ in range(STEPS):
        grad=finite_diff(circuit,p)
        p-=LR*grad
    return time.time()-t0

def run(N):
    print(f"\n=== {N}-QUBIT VQC TRAINING ({STEPS} steps) ===")
    params=np.random.uniform(-0.1,0.1,2*N)

    t_qcpu=train_qulacs_cpu(N, params)
    print(f"Qulacs CPU:             {t_qcpu:.4f} s")

    t_qgpu=train_qulacs_gpu(N, params)
    print(f"Qulacs GPU:             {t_qgpu:.4f} s")

    if N <= 12:
        t_pdef=train_pl_default(N, params)
        print(f"PL default.qubit CPU:   {t_pdef:.4f} s")

        t_plight=train_pl_lightning(N, params)
        print(f"PL lightning.qubit CPU: {t_plight:.4f} s")
    else:
        print("PennyLane >12 qubits skipped (too slow).")

    print("\nSUMMARY:")
    print(f" CPU Qulacs:  {t_qcpu:.4f}s")
    print(f" GPU Qulacs:  {t_qgpu:.4f}s")

# RUN
run(12)
run(14)
run(16)


=== 12-QUBIT VQC TRAINING (25 steps) ===
Qulacs CPU:             0.2264 s
Qulacs GPU:             0.4384 s
PL default.qubit CPU:   4.2699 s
PL lightning.qubit CPU: 1.8945 s

SUMMARY:
 CPU Qulacs:  0.2264s
 GPU Qulacs:  0.4384s

=== 14-QUBIT VQC TRAINING (25 steps) ===
Qulacs CPU:             0.5225 s
Qulacs GPU:             0.7105 s
PennyLane >12 qubits skipped (too slow).

SUMMARY:
 CPU Qulacs:  0.5225s
 GPU Qulacs:  0.7105s

=== 16-QUBIT VQC TRAINING (25 steps) ===
Qulacs CPU:             1.8714 s
Qulacs GPU:             2.0065 s
PennyLane >12 qubits skipped (too slow).

SUMMARY:
 CPU Qulacs:  1.8714s
 GPU Qulacs:  2.0065s


In [1]:
import numpy as np
import time
from qulacs import QuantumCircuit, QuantumState, QuantumStateGpu, Observable

# =========================
# Config
# =========================
N = 16              # qubits
DEPTH = 40          # how many layers (heavier = bigger)
STEPS = 10          # training steps (each is very expensive now)
LR = 0.05
EPS = 1e-4
TARGET = 1.0

np.random.seed(0)
INIT_PARAMS = np.random.uniform(-0.1, 0.1, size=2 * N * DEPTH)  # params per layer

# =========================
# Common observable (Z0)
# =========================
obs = Observable(N)
obs.add_operator(1.0, "Z 0")

# =========================
# Build deep ansatz
# =========================
def build_ansatz(N, params):
    qc = QuantumCircuit(N)
    idx = 0
    for _ in range(DEPTH):
        # single-qubit rotations
        for q in range(N):
            qc.add_RX_gate(q, params[idx]); idx += 1
            qc.add_RY_gate(q, params[idx]); idx += 1
        # entangling ring
        for q in range(N - 1):
            qc.add_CNOT_gate(q, q + 1)
        qc.add_CNOT_gate(N - 1, 0)
    return qc

# =========================
# Finite-diff gradient
# =========================
def finite_diff(eval_fn, params):
    grad = np.zeros_like(params)
    for i in range(len(params)):
        p1 = params.copy(); p1[i] += EPS
        p2 = params.copy(); p2[i] -= EPS
        grad[i] = (eval_fn(p1) - eval_fn(p2)) / (2 * EPS)
    return grad

# =========================
# Qulacs CPU
# =========================
def eval_cpu(params):
    qc = build_ansatz(N, params)
    st = QuantumState(N)
    st.set_zero_state()
    qc.update_quantum_state(st)
    return obs.get_expectation_value(st)

def train_qulacs_cpu():
    params = INIT_PARAMS.copy()
    t0 = time.time()
    for step in range(STEPS):
        val = eval_cpu(params)
        loss = (TARGET - val) ** 2
        grad = finite_diff(eval_cpu, params)
        params -= LR * grad
        print(f"[CPU] step {step:2d} | loss={loss:.4f} | <Z0>={val:.4f}")
    return time.time() - t0

# =========================
# Qulacs GPU
# =========================
def eval_gpu(params):
    qc = build_ansatz(N, params)
    st = QuantumStateGpu(N)
    st.set_zero_state()
    qc.update_quantum_state(st)
    return obs.get_expectation_value(st)

def train_qulacs_gpu():
    params = INIT_PARAMS.copy()
    t0 = time.time()
    for step in range(STEPS):
        val = eval_gpu(params)
        loss = (TARGET - val) ** 2
        grad = finite_diff(eval_gpu, params)
        params -= LR * grad
        print(f"[GPU] step {step:2d} | loss={loss:.4f} | <Z0>={val:.4f}")
    return time.time() - t0

# =========================
# Run both
# =========================
print(f"=== DEEP VQC TRAINING: {N} qubits, depth={DEPTH}, steps={STEPS} ===\n")

cpu_time = train_qulacs_cpu()
print(f"\nCPU training time: {cpu_time:.3f} s\n")

gpu_time = train_qulacs_gpu()
print(f"\nGPU training time: {gpu_time:.3f} s\n")

print("=== SUMMARY ===")
print(f"Qulacs CPU total: {cpu_time:.3f} s")
print(f"Qulacs GPU total: {gpu_time:.3f} s")
print(f"Speedup (CPU / GPU): {cpu_time / gpu_time:.2f}x")

=== DEEP VQC TRAINING: 16 qubits, depth=40, steps=10 ===

[CPU] step  0 | loss=0.2699 | <Z0>=0.4804
[CPU] step  1 | loss=0.3174 | <Z0>=0.4366
[CPU] step  2 | loss=0.3676 | <Z0>=0.3937
[CPU] step  3 | loss=0.4192 | <Z0>=0.3525
[CPU] step  4 | loss=0.4712 | <Z0>=0.3135
[CPU] step  5 | loss=0.5225 | <Z0>=0.2772
[CPU] step  6 | loss=0.5721 | <Z0>=0.2436
[CPU] step  7 | loss=0.6195 | <Z0>=0.2129
[CPU] step  8 | loss=0.6640 | <Z0>=0.1851
[CPU] step  9 | loss=0.7055 | <Z0>=0.1601

CPU training time: 463.119 s

[GPU] step  0 | loss=0.2699 | <Z0>=0.4804
[GPU] step  1 | loss=0.3174 | <Z0>=0.4366
[GPU] step  2 | loss=0.3676 | <Z0>=0.3937
[GPU] step  3 | loss=0.4192 | <Z0>=0.3525
[GPU] step  4 | loss=0.4712 | <Z0>=0.3135
[GPU] step  5 | loss=0.5225 | <Z0>=0.2772
[GPU] step  6 | loss=0.5721 | <Z0>=0.2436
[GPU] step  7 | loss=0.6195 | <Z0>=0.2129
[GPU] step  8 | loss=0.6640 | <Z0>=0.1851
[GPU] step  9 | loss=0.7055 | <Z0>=0.1601

GPU training time: 958.777 s

=== SUMMARY ===
Qulacs CPU total: 463.11

In [1]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
import time

def benchmark(backend_name):
    backend = Aer.get_backend(backend_name)
    qc = QuantumCircuit(20)
    qc.h(range(20))
    for i in range(19):
        qc.cx(i, i+1)
    qc.measure_all()

    start = time.time()
    result = backend.run(qc, shots=200).result()
    return time.time() - start

print("Running CPU vs GPU benchmark…")

cpu_time = benchmark("aer_simulator_statevector")
gpu_time = benchmark("aer_simulator_statevector_gpu")

print("\n=== RESULTS ===")
print(f"CPU time: {cpu_time:.4f} s")
print(f"GPU time: {gpu_time:.4f} s")
print(f"Speedup: {cpu_time/gpu_time:.2f}x")

Running CPU vs GPU benchmark…

=== RESULTS ===
CPU time: 0.0891 s
GPU time: 0.0748 s
Speedup: 1.19x


In [4]:
import time
import numpy as np

# Qulacs
from qulacs import QuantumCircuit, QuantumState, QuantumStateGpu, Observable

# Qiskit
from qiskit import QuantumCircuit as QiskitCircuit
from qiskit.circuit import ParameterVector
from qiskit_aer import Aer

# -----------------------------
# Config
# -----------------------------
N_QUBITS = 6
DEPTH = 3
LR = 0.2
STEPS = 10   # increase later if you want heavier training

np.random.seed(0)

N_PARAMS = DEPTH * N_QUBITS  # one RY per qubit per layer
initial_params = np.random.uniform(-np.pi, np.pi, size=N_PARAMS)


# =============================
# 1) QULACS IMPLEMENTATION
# =============================

def build_qulacs_circuit(params):
    """Builds a Qulacs VQC circuit with given params."""
    circ = QuantumCircuit(N_QUBITS)
    idx = 0
    for d in range(DEPTH):
        # RY layer
        for q in range(N_QUBITS):
            circ.add_RX_gate(q, params[idx])  # or RY_gate, but RX is fine
            idx += 1
        # CZ entangling layer
        for q in range(N_QUBITS - 1):
            circ.add_CZ_gate(q, q + 1)
    return circ

def qulacs_expectation(params, use_gpu=False):
    """Compute <Z0> expectation with Qulacs (CPU or GPU)."""
    if use_gpu:
        state = QuantumStateGpu(N_QUBITS)
    else:
        state = QuantumState(N_QUBITS)
    state.set_zero_state()

    circ = build_qulacs_circuit(params)
    circ.update_quantum_state(state)

    obs = Observable(N_QUBITS)
    obs.add_operator(1.0, "Z 0")
    return obs.get_expectation_value(state)

def qulacs_train(use_gpu=False, steps=STEPS, lr=LR):
    params = initial_params.copy()
    start = time.time()

    for step in range(steps):
        # parameter-shift gradient
        grad = np.zeros_like(params)
        base_expect = qulacs_expectation(params, use_gpu=use_gpu)

        for i in range(len(params)):
            shift = np.zeros_like(params)
            shift[i] = np.pi / 2

            plus = qulacs_expectation(params + shift, use_gpu=use_gpu)
            minus = qulacs_expectation(params - shift, use_gpu=use_gpu)
            grad[i] = 0.5 * (plus - minus)  # param-shift rule

        # loss = (target - exp)^2, target=-1 => grad_loss = 2*(exp+1)*grad_exp
        loss_grad_factor = 2 * (base_expect + 1.0)
        grad *= loss_grad_factor

        params -= lr * grad

        if step % max(1, steps // 5) == 0 or step == steps - 1:
            loss = (base_expect + 1.0) ** 2
            print(f"[Qulacs {'GPU' if use_gpu else 'CPU'}] step {step:2d} | "
                  f"loss={loss:.4f} | <Z0>={base_expect:.4f}")

    total_time = time.time() - start
    final_expect = qulacs_expectation(params, use_gpu=use_gpu)
    return total_time, final_expect


# =============================
# 2) QISKIT + AER IMPLEMENTATION
# =============================

def build_qiskit_circuit(params):
    """Builds a Qiskit VQC with numeric parameters."""
    qc = QiskitCircuit(N_QUBITS)
    idx = 0
    for d in range(DEPTH):
        for q in range(N_QUBITS):
            qc.rx(float(params[idx]), q)
            idx += 1
        for q in range(N_QUBITS - 1):
            qc.cz(q, q + 1)
    qc.save_statevector()
    return qc

def z0_expect_from_statevector(statevector):
    """Compute <Z0> from a statevector array."""
    exp = 0.0
    for idx, amp in enumerate(statevector):
        prob = abs(amp) ** 2
        # bit 0 is least significant bit
        bit0 = (idx & 1)
        z_val = 1.0 if bit0 == 0 else -1.0
        exp += prob * z_val
    return exp

def qiskit_expectation(params, backend_name):
    backend = Aer.get_backend(backend_name)
    qc = build_qiskit_circuit(params)
    result = backend.run(qc).result()
    statevector = result.get_statevector(qc)
    return z0_expect_from_statevector(statevector)

def qiskit_train(backend_name, steps=STEPS, lr=LR):
    params = initial_params.copy()
    start = time.time()

    for step in range(steps):
        base_expect = qiskit_expectation(params, backend_name)

        # param-shift gradient
        grad = np.zeros_like(params)
        for i in range(len(params)):
            shift = np.zeros_like(params)
            shift[i] = np.pi / 2
            plus = qiskit_expectation(params + shift, backend_name)
            minus = qiskit_expectation(params - shift, backend_name)
            grad[i] = 0.5 * (plus - minus)

        loss_grad_factor = 2 * (base_expect + 1.0)
        grad *= loss_grad_factor
        params -= lr * grad

        if step % max(1, steps // 5) == 0 or step == steps - 1:
            loss = (base_expect + 1.0) ** 2
            print(f"[Qiskit {backend_name}] step {step:2d} | "
                  f"loss={loss:.4f} | <Z0>={base_expect:.4f}")

    total_time = time.time() - start
    final_expect = qiskit_expectation(params, backend_name)
    return total_time, final_expect


# =============================
# 3) RUN ALL FOUR BENCHMARKS
# =============================

print("=== VQC TRAINING BENCHMARK ===")
print(f"Qubits: {N_QUBITS}, Depth: {DEPTH}, Steps: {STEPS}\n")

print("-> Qulacs CPU")
t_qulacs_cpu, e_qulacs_cpu = qulacs_train(use_gpu=False)
print("\n-> Qulacs GPU")
t_qulacs_gpu, e_qulacs_gpu = qulacs_train(use_gpu=True)

print("\n-> Qiskit Aer CPU (statevector)")
t_qiskit_cpu, e_qiskit_cpu = qiskit_train("aer_simulator_statevector")

print("\n-> Qiskit Aer GPU (statevector)")
t_qiskit_gpu, e_qiskit_gpu = qiskit_train("aer_simulator_statevector_gpu")

print("\n=== SUMMARY (VQC training) ===")
print(f"Qulacs CPU:              {t_qulacs_cpu:.3f} s   final <Z0>={e_qulacs_cpu:.4f}")
print(f"Qulacs GPU:              {t_qulacs_gpu:.3f} s   final <Z0>={e_qulacs_gpu:.4f}")
print(f"Qiskit Aer CPU:          {t_qiskit_cpu:.3f} s   final <Z0>={e_qiskit_cpu:.4f}")
print(f"Qiskit Aer GPU:          {t_qiskit_gpu:.3f} s   final <Z0>={e_qiskit_gpu:.4f}")

print("\nSpeedup Qulacs CPU / Qulacs GPU:   {:.2f}x".format(t_qulacs_cpu / t_qulacs_gpu))
print("Speedup Qiskit CPU / Qiskit GPU:   {:.2f}x".format(t_qiskit_cpu / t_qiskit_gpu))

=== VQC TRAINING BENCHMARK ===
Qubits: 6, Depth: 3, Steps: 10

-> Qulacs CPU
[Qulacs CPU] step  0 | loss=3.2805 | <Z0>=0.8112
[Qulacs CPU] step  2 | loss=0.1897 | <Z0>=-0.5644
[Qulacs CPU] step  4 | loss=0.0329 | <Z0>=-0.8185
[Qulacs CPU] step  6 | loss=0.0149 | <Z0>=-0.8779
[Qulacs CPU] step  8 | loss=0.0088 | <Z0>=-0.9060
[Qulacs CPU] step  9 | loss=0.0072 | <Z0>=-0.9152

-> Qulacs GPU
[Qulacs GPU] step  0 | loss=3.2805 | <Z0>=0.8112
[Qulacs GPU] step  2 | loss=0.1897 | <Z0>=-0.5644
[Qulacs GPU] step  4 | loss=0.0329 | <Z0>=-0.8185
[Qulacs GPU] step  6 | loss=0.0149 | <Z0>=-0.8779
[Qulacs GPU] step  8 | loss=0.0088 | <Z0>=-0.9060
[Qulacs GPU] step  9 | loss=0.0072 | <Z0>=-0.9152

-> Qiskit Aer CPU (statevector)
[Qiskit aer_simulator_statevector] step  0 | loss=3.2805 | <Z0>=0.8112
[Qiskit aer_simulator_statevector] step  2 | loss=0.1897 | <Z0>=-0.5644


/tmp/ipykernel_58599/555519835.py:112: DeprecationWarning: The return type of saved statevectors has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.Statevector` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
  for idx, amp in enumerate(statevector):


[Qiskit aer_simulator_statevector] step  4 | loss=0.0329 | <Z0>=-0.8185
[Qiskit aer_simulator_statevector] step  6 | loss=0.0149 | <Z0>=-0.8779
[Qiskit aer_simulator_statevector] step  8 | loss=0.0088 | <Z0>=-0.9060
[Qiskit aer_simulator_statevector] step  9 | loss=0.0072 | <Z0>=-0.9152

-> Qiskit Aer GPU (statevector)
[Qiskit aer_simulator_statevector_gpu] step  0 | loss=3.2805 | <Z0>=0.8112
[Qiskit aer_simulator_statevector_gpu] step  2 | loss=0.1897 | <Z0>=-0.5644
[Qiskit aer_simulator_statevector_gpu] step  4 | loss=0.0329 | <Z0>=-0.8185
[Qiskit aer_simulator_statevector_gpu] step  6 | loss=0.0149 | <Z0>=-0.8779
[Qiskit aer_simulator_statevector_gpu] step  8 | loss=0.0088 | <Z0>=-0.9060
[Qiskit aer_simulator_statevector_gpu] step  9 | loss=0.0072 | <Z0>=-0.9152

=== SUMMARY (VQC training) ===
Qulacs CPU:              0.203 s   final <Z0>=-0.9227
Qulacs GPU:              0.475 s   final <Z0>=-0.9227
Qiskit Aer CPU:          0.307 s   final <Z0>=-0.9227
Qiskit Aer GPU:          0.405

In [5]:
import time
import numpy as np

# Qulacs
from qulacs import QuantumCircuit, QuantumState, QuantumStateGpu, Observable

# Qiskit
from qiskit import QuantumCircuit as QiskitCircuit
from qiskit.circuit import ParameterVector
from qiskit_aer import Aer

# -----------------------------
# Config
# -----------------------------
N_QUBITS = 18
DEPTH = 5
LR = 0.01
STEPS = 10   # increase later if you want heavier training

np.random.seed(0)

N_PARAMS = DEPTH * N_QUBITS  # one RY per qubit per layer
initial_params = np.random.uniform(-np.pi, np.pi, size=N_PARAMS)


# =============================
# 1) QULACS IMPLEMENTATION
# =============================

def build_qulacs_circuit(params):
    """Builds a Qulacs VQC circuit with given params."""
    circ = QuantumCircuit(N_QUBITS)
    idx = 0
    for d in range(DEPTH):
        # RY layer
        for q in range(N_QUBITS):
            circ.add_RX_gate(q, params[idx])  # or RY_gate, but RX is fine
            idx += 1
        # CZ entangling layer
        for q in range(N_QUBITS - 1):
            circ.add_CZ_gate(q, q + 1)
    return circ

def qulacs_expectation(params, use_gpu=False):
    """Compute <Z0> expectation with Qulacs (CPU or GPU)."""
    if use_gpu:
        state = QuantumStateGpu(N_QUBITS)
    else:
        state = QuantumState(N_QUBITS)
    state.set_zero_state()

    circ = build_qulacs_circuit(params)
    circ.update_quantum_state(state)

    obs = Observable(N_QUBITS)
    obs.add_operator(1.0, "Z 0")
    return obs.get_expectation_value(state)

def qulacs_train(use_gpu=False, steps=STEPS, lr=LR):
    params = initial_params.copy()
    start = time.time()

    for step in range(steps):
        # parameter-shift gradient
        grad = np.zeros_like(params)
        base_expect = qulacs_expectation(params, use_gpu=use_gpu)

        for i in range(len(params)):
            shift = np.zeros_like(params)
            shift[i] = np.pi / 2

            plus = qulacs_expectation(params + shift, use_gpu=use_gpu)
            minus = qulacs_expectation(params - shift, use_gpu=use_gpu)
            grad[i] = 0.5 * (plus - minus)  # param-shift rule

        # loss = (target - exp)^2, target=-1 => grad_loss = 2*(exp+1)*grad_exp
        loss_grad_factor = 2 * (base_expect + 1.0)
        grad *= loss_grad_factor

        params -= lr * grad

        if step % max(1, steps // 5) == 0 or step == steps - 1:
            loss = (base_expect + 1.0) ** 2
            print(f"[Qulacs {'GPU' if use_gpu else 'CPU'}] step {step:2d} | "
                  f"loss={loss:.4f} | <Z0>={base_expect:.4f}")

    total_time = time.time() - start
    final_expect = qulacs_expectation(params, use_gpu=use_gpu)
    return total_time, final_expect


# =============================
# 2) QISKIT + AER IMPLEMENTATION
# =============================

def build_qiskit_circuit(params):
    """Builds a Qiskit VQC with numeric parameters."""
    qc = QiskitCircuit(N_QUBITS)
    idx = 0
    for d in range(DEPTH):
        for q in range(N_QUBITS):
            qc.rx(float(params[idx]), q)
            idx += 1
        for q in range(N_QUBITS - 1):
            qc.cz(q, q + 1)
    qc.save_statevector()
    return qc

def z0_expect_from_statevector(statevector):
    """Compute <Z0> from a statevector array."""
    exp = 0.0
    for idx, amp in enumerate(statevector):
        prob = abs(amp) ** 2
        # bit 0 is least significant bit
        bit0 = (idx & 1)
        z_val = 1.0 if bit0 == 0 else -1.0
        exp += prob * z_val
    return exp

def qiskit_expectation(params, backend_name):
    backend = Aer.get_backend(backend_name)
    qc = build_qiskit_circuit(params)
    result = backend.run(qc).result()
    statevector = result.get_statevector(qc)
    return z0_expect_from_statevector(statevector)

def qiskit_train(backend_name, steps=STEPS, lr=LR):
    params = initial_params.copy()
    start = time.time()

    for step in range(steps):
        base_expect = qiskit_expectation(params, backend_name)

        # param-shift gradient
        grad = np.zeros_like(params)
        for i in range(len(params)):
            shift = np.zeros_like(params)
            shift[i] = np.pi / 2
            plus = qiskit_expectation(params + shift, backend_name)
            minus = qiskit_expectation(params - shift, backend_name)
            grad[i] = 0.5 * (plus - minus)

        loss_grad_factor = 2 * (base_expect + 1.0)
        grad *= loss_grad_factor
        params -= lr * grad

        if step % max(1, steps // 5) == 0 or step == steps - 1:
            loss = (base_expect + 1.0) ** 2
            print(f"[Qiskit {backend_name}] step {step:2d} | "
                  f"loss={loss:.4f} | <Z0>={base_expect:.4f}")

    total_time = time.time() - start
    final_expect = qiskit_expectation(params, backend_name)
    return total_time, final_expect


# =============================
# 3) RUN ALL FOUR BENCHMARKS
# =============================

print("=== VQC TRAINING BENCHMARK ===")
print(f"Qubits: {N_QUBITS}, Depth: {DEPTH}, Steps: {STEPS}\n")

print("-> Qulacs CPU")
t_qulacs_cpu, e_qulacs_cpu = qulacs_train(use_gpu=False)
print("\n-> Qulacs GPU")
t_qulacs_gpu, e_qulacs_gpu = qulacs_train(use_gpu=True)

print("\n-> Qiskit Aer CPU (statevector)")
t_qiskit_cpu, e_qiskit_cpu = qiskit_train("aer_simulator_statevector")

print("\n-> Qiskit Aer GPU (statevector)")
t_qiskit_gpu, e_qiskit_gpu = qiskit_train("aer_simulator_statevector_gpu")

print("\n=== SUMMARY (VQC training) ===")
print(f"Qulacs CPU:              {t_qulacs_cpu:.3f} s   final <Z0>={e_qulacs_cpu:.4f}")
print(f"Qulacs GPU:              {t_qulacs_gpu:.3f} s   final <Z0>={e_qulacs_gpu:.4f}")
print(f"Qiskit Aer CPU:          {t_qiskit_cpu:.3f} s   final <Z0>={e_qiskit_cpu:.4f}")
print(f"Qiskit Aer GPU:          {t_qiskit_gpu:.3f} s   final <Z0>={e_qiskit_gpu:.4f}")

print("\nSpeedup Qulacs CPU / Qulacs GPU:   {:.2f}x".format(t_qulacs_cpu / t_qulacs_gpu))
print("Speedup Qiskit CPU / Qiskit GPU:   {:.2f}x".format(t_qiskit_cpu / t_qiskit_gpu))

=== VQC TRAINING BENCHMARK ===
Qubits: 18, Depth: 5, Steps: 10

-> Qulacs CPU
[Qulacs CPU] step  0 | loss=0.2277 | <Z0>=-0.5228
[Qulacs CPU] step  2 | loss=0.1921 | <Z0>=-0.5617
[Qulacs CPU] step  4 | loss=0.1637 | <Z0>=-0.5954
[Qulacs CPU] step  6 | loss=0.1408 | <Z0>=-0.6248
[Qulacs CPU] step  8 | loss=0.1221 | <Z0>=-0.6505
[Qulacs CPU] step  9 | loss=0.1141 | <Z0>=-0.6622

-> Qulacs GPU
[Qulacs GPU] step  0 | loss=0.2277 | <Z0>=-0.5228
[Qulacs GPU] step  2 | loss=0.1921 | <Z0>=-0.5617
[Qulacs GPU] step  4 | loss=0.1637 | <Z0>=-0.5954
[Qulacs GPU] step  6 | loss=0.1408 | <Z0>=-0.6248
[Qulacs GPU] step  8 | loss=0.1221 | <Z0>=-0.6505
[Qulacs GPU] step  9 | loss=0.1141 | <Z0>=-0.6622

-> Qiskit Aer CPU (statevector)


/tmp/ipykernel_58599/3504778765.py:112: DeprecationWarning: The return type of saved statevectors has been changed from a `numpy.ndarray` to a `qiskit.quantum_info.Statevector` as of qiskit-aer 0.10. Accessing numpy array attributes is deprecated and will result in an error in a future release. To continue using saved result objects as arrays you can explicitly cast them using  `np.asarray(object)`.
  for idx, amp in enumerate(statevector):


[Qiskit aer_simulator_statevector] step  0 | loss=0.2277 | <Z0>=-0.5228
[Qiskit aer_simulator_statevector] step  2 | loss=0.1921 | <Z0>=-0.5617
[Qiskit aer_simulator_statevector] step  4 | loss=0.1637 | <Z0>=-0.5954
[Qiskit aer_simulator_statevector] step  6 | loss=0.1408 | <Z0>=-0.6248
[Qiskit aer_simulator_statevector] step  8 | loss=0.1221 | <Z0>=-0.6505
[Qiskit aer_simulator_statevector] step  9 | loss=0.1141 | <Z0>=-0.6622

-> Qiskit Aer GPU (statevector)
[Qiskit aer_simulator_statevector_gpu] step  0 | loss=0.2277 | <Z0>=-0.5228
[Qiskit aer_simulator_statevector_gpu] step  2 | loss=0.1921 | <Z0>=-0.5617
[Qiskit aer_simulator_statevector_gpu] step  4 | loss=0.1637 | <Z0>=-0.5954
[Qiskit aer_simulator_statevector_gpu] step  6 | loss=0.1408 | <Z0>=-0.6248
[Qiskit aer_simulator_statevector_gpu] step  8 | loss=0.1221 | <Z0>=-0.6505
[Qiskit aer_simulator_statevector_gpu] step  9 | loss=0.1141 | <Z0>=-0.6622

=== SUMMARY (VQC training) ===
Qulacs CPU:              11.018 s   final <Z0>=